# 감성 분류 모델 만들기 (긍정, 부정)

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
# 테스트 문장과 레이블(긍정:1, 부정:0) 생성

sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

## 전처리, 토큰화 Tokenizer()

In [13]:
# 전처리
# 토큰화

t = Tokenizer()
# fit_on_texts() 메서드는 문자 데이터를 입력받아서 리스트의 형태로 변환
t.fit_on_texts(sentences)
# okenizer의 word_index 속성은 단어와 숫자의 키-값 쌍을 포함하는 딕셔너리를 반환
vocab_size = len(t.word_index) + 1 # 0값인 패딩이 있기때문에 +1추가

print(vocab_size)
print(t.word_index)

16
{'nice': 1, 'great': 2, 'best': 3, 'amazing': 4, 'stop': 5, 'lies': 6, 'pitiful': 7, 'nerd': 8, 'excellent': 9, 'work': 10, 'supreme': 11, 'quality': 12, 'bad': 13, 'highly': 14, 'respectable': 15}


## 정수 인코딩 texts_to_sequences()

In [4]:
# 정수 인코딩
# texts_to_sequences() 메서드를 이용해서 이러한 단어들을 시퀀스의 형태로 변환
X_encoded = t.texts_to_sequences(sentences)
print(X_encoded)

[[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


In [5]:
# 패딩을 위해 문장 중 가장 길이가 긴 문장의 길이 확인
max_len = max(len(l) for l in X_encoded)
print(max_len)

4


## 패딩 pad_sequences()

In [8]:
# 패딩
# 비어있는 자리는 0으로 채움
X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train = np.array(y_train)
print(X_train)
print()
print(y_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]

[1 0 0 1 1 0 1]


In [9]:
print(y_train)

[1 0 0 1 1 0 1]


## Embedding 모델 설계

In [16]:
# 모델 설계
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len))  # 모든 임베딩 벡터는 4차원 
                                                           # vocab_size 전체 단어 집합의 크기 (16)
                                                           # output_dim : 워드 임베딩 후의 임베딩 벡터의 차원
                                                           # input_length : 입력 시퀀스의 길이
model.add(Flatten())   # Dense의 입력으로 넣기 위함
model.add(Dense(1, activation='sigmoid'))

## 모델 훈련

In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 1s - loss: 0.6935 - acc: 0.5714
Epoch 2/100
1/1 - 0s - loss: 0.6925 - acc: 0.5714
Epoch 3/100
1/1 - 0s - loss: 0.6914 - acc: 0.5714
Epoch 4/100
1/1 - 0s - loss: 0.6904 - acc: 0.5714
Epoch 5/100
1/1 - 0s - loss: 0.6894 - acc: 0.5714
Epoch 6/100
1/1 - 0s - loss: 0.6883 - acc: 0.5714
Epoch 7/100
1/1 - 0s - loss: 0.6873 - acc: 0.5714
Epoch 8/100
1/1 - 0s - loss: 0.6862 - acc: 0.5714
Epoch 9/100
1/1 - 0s - loss: 0.6852 - acc: 0.5714
Epoch 10/100
1/1 - 0s - loss: 0.6841 - acc: 0.5714
Epoch 11/100
1/1 - 0s - loss: 0.6831 - acc: 0.5714
Epoch 12/100
1/1 - 0s - loss: 0.6820 - acc: 0.5714
Epoch 13/100
1/1 - 0s - loss: 0.6810 - acc: 0.7143
Epoch 14/100
1/1 - 0s - loss: 0.6799 - acc: 0.7143
Epoch 15/100
1/1 - 0s - loss: 0.6789 - acc: 0.7143
Epoch 16/100
1/1 - 0s - loss: 0.6778 - acc: 0.7143
Epoch 17/100
1/1 - 0s - loss: 0.6768 - acc: 0.7143
Epoch 18/100
1/1 - 0s - loss: 0.6757 - acc: 0.7143
Epoch 19/100
1/1 - 0s - loss: 0.6746 - acc: 0.7143
Epoch 20/100
1/1 - 0s - loss: 0.6735 - a

# Pretrained Word Embedding Model

  - Word2Vec

  [Pre-trained word vectors of 30+ languages](https://github.com/Kyubyong/wordvectors)

  - FastText

  [Word vectors for 157 languages](https://fasttext.cc/docs/en/crawl-vectors.html)

## 1) 사전 훈련된 GloVe 사용하기

In [18]:
# 데이터 준비하기

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

# 하나의 줄당 101개의 값을 가지는 리스트 갖고 있음

--2021-09-09 05:57:49--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-09-09 05:57:49--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-09-09 05:57:49--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [19]:
# 두 개의 줄만 읽어 보기

n = 0
f = open('glove.6B.100d.txt', encoding='utf8') # 100차원 파일열기

for line in f:
    word_vector = line.split()  # 각 줄을 읽어와서 word_vector에 저장 (임베딩 벡터로 이루어진 데이터)
    print(word_vector)  # 각 줄을 출력
    word = word_vector[0]  # word_vector에서 첫번째 값만 저장 (the)
    print(word)  # word_vector의 첫번째 값만 출력
    n = n+1
    if n == 2:
        break
f.close()

# 101개의 값 중 첫번째 값 'the', ',': 임베딩 벡터가 의미하는 단어, 이후 임베딩 벡처 100개의 차원의 각각의 값

['the', '-0.038194', '-0.24487', '0.72812', '-0.39961', '0.083172', '0.043953', '-0.39141', '0.3344', '-0.57545', '0.087459', '0.28787', '-0.06731', '0.30906', '-0.26384', '-0.13231', '-0.20757', '0.33395', '-0.33848', '-0.31743', '-0.48336', '0.1464', '-0.37304', '0.34577', '0.052041', '0.44946', '-0.46971', '0.02628', '-0.54155', '-0.15518', '-0.14107', '-0.039722', '0.28277', '0.14393', '0.23464', '-0.31021', '0.086173', '0.20397', '0.52624', '0.17164', '-0.082378', '-0.71787', '-0.41531', '0.20335', '-0.12763', '0.41367', '0.55187', '0.57908', '-0.33477', '-0.36559', '-0.54857', '-0.062892', '0.26584', '0.30205', '0.99775', '-0.80481', '-3.0243', '0.01254', '-0.36942', '2.2167', '0.72201', '-0.24978', '0.92136', '0.034514', '0.46745', '1.1079', '-0.19358', '-0.074575', '0.23353', '-0.052062', '-0.22044', '0.057162', '-0.15806', '-0.30798', '-0.41625', '0.37972', '0.15006', '-0.53212', '-0.2055', '-1.2526', '0.071624', '0.70565', '0.49744', '-0.42063', '0.26148', '-1.538', '-0.30223

In [20]:
print(type(word_vector))
print(len(word_vector))

<class 'list'>
101


In [21]:
# 모든 임베딩 벡터들을 dictionary type(key:value)으로 불러오기
embedding_dict = dict()

f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()
print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

400000개의 Embedding vector가 있습니다.


In [22]:
# 임의의 단어 'respectable'에 대해서 임베딩 벡터를 출력 해보기
# 각각의 벡터 값들과 총 개수 출력
print(embedding_dict['respectable'])
print(len(embedding_dict['respectable']))

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

In [27]:
# 훈련 데이터의 단어 집합의 모든 단어에 대해서 사전 훈련된 GloVe의 임베딩 벡터들을 맵핑
embedding_matrix = np.zeros((vocab_size, 100))
# 단어 집합 크기의 행과 100개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

(16, 100)

In [28]:
print(t.word_index.items())

dict_items([('nice', 1), ('great', 2), ('best', 3), ('amazing', 4), ('stop', 5), ('lies', 6), ('pitiful', 7), ('nerd', 8), ('excellent', 9), ('work', 10), ('supreme', 11), ('quality', 12), ('bad', 13), ('highly', 14), ('respectable', 15)])


In [29]:
for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어를 1개씩 꺼내온다.
    temp = embedding_dict.get(word) # 단어(key) 해당되는 임베딩 벡터의 100개의 값(value)를 임시 변수에 저장
    if temp is not None: # 임수 변수의 값을 단어와 맵핑되는 인덱스의 행에 삽입
        embedding_matrix[i] = temp

In [31]:
embedding_matrix[1]

array([-0.18554001,  0.047152  ,  0.34867001, -0.23114   , -0.26082999,
        0.63107997,  0.55686998,  0.61622   , -0.15436999, -0.38381001,
        0.12445   , -0.26999   , -0.29196   ,  0.1125    ,  0.36035001,
        0.70688999, -0.33891001, -0.26949999,  0.17481001,  0.97048002,
        0.23014   ,  0.63168001, -0.24542999, -0.72890002,  0.32517001,
       -0.21118   , -0.80353999, -0.59863001, -0.10182   , -0.87826002,
       -0.80162001,  0.20998999,  0.64598   , -0.38238999,  0.64512002,
        0.73045999, -0.19881   ,  0.35716999,  0.19135   , -0.43686   ,
        0.75955999, -0.66430998,  0.34509   , -1.03830004, -0.50490999,
        0.19976   , -0.041208  ,  0.16952001,  0.17821001, -0.84249002,
        0.21991999, -0.16474   , -0.24669001,  0.34117001, -0.59713   ,
       -2.3434    ,  0.31483999,  0.69668001,  0.53368002, -0.62857002,
       -0.197     ,  0.52240998, -1.59029996, -0.16474999,  0.62553   ,
       -0.094116  ,  0.0070705 ,  0.22617   , -0.45697999, -0.53

In [32]:
# 임베딩 층 생성 : 모델 설계
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)
                                                         # input_length : 입력 시퀀스의 길이
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [33]:
print(max_len)

4


In [34]:
# 모델 훈련
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.7125 - acc: 0.2857
Epoch 2/100
1/1 - 0s - loss: 0.6912 - acc: 0.2857
Epoch 3/100
1/1 - 0s - loss: 0.6705 - acc: 0.4286
Epoch 4/100
1/1 - 0s - loss: 0.6507 - acc: 0.4286
Epoch 5/100
1/1 - 0s - loss: 0.6317 - acc: 0.5714
Epoch 6/100
1/1 - 0s - loss: 0.6134 - acc: 0.5714
Epoch 7/100
1/1 - 0s - loss: 0.5958 - acc: 0.7143
Epoch 8/100
1/1 - 0s - loss: 0.5790 - acc: 0.7143
Epoch 9/100
1/1 - 0s - loss: 0.5629 - acc: 0.7143
Epoch 10/100
1/1 - 0s - loss: 0.5474 - acc: 0.7143
Epoch 11/100
1/1 - 0s - loss: 0.5325 - acc: 0.7143
Epoch 12/100
1/1 - 0s - loss: 0.5183 - acc: 0.7143
Epoch 13/100
1/1 - 0s - loss: 0.5046 - acc: 0.8571
Epoch 14/100
1/1 - 0s - loss: 0.4913 - acc: 0.8571
Epoch 15/100
1/1 - 0s - loss: 0.4786 - acc: 0.8571
Epoch 16/100
1/1 - 0s - loss: 0.4664 - acc: 0.8571
Epoch 17/100
1/1 - 0s - loss: 0.4545 - acc: 1.0000
Epoch 18/100
1/1 - 0s - loss: 0.4431 - acc: 1.0000
Epoch 19/100
1/1 - 0s - loss: 0.4320 - acc: 1.0000
Epoch 20/100
1/1 - 0s - loss: 0.4213 - a

## 2) 사전 훈련된 Word2Vec 사용하기

In [35]:
import gensim

# 데이터 준비하기
# 현재 위치에 구글의 사전훈련된 Word2Vec을 다운로드
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-09-09 06:22:41--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.134.40
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.134.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  67.1MB/s    in 29s     

2021-09-09 06:23:10 (55.0 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [36]:
# 구글의 사전 훈련된 Word2vec 모델 로드
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [37]:
# 모델 크기 확인
print(word2vec_model.vectors.shape)

(3000000, 300)


In [38]:
# 임베딩 행렬 생성 : 모두 0으로(이번 문제의 단어는 총 16개이므로, 16 × 300의 크기를 가진 행렬)
embedding_matrix = np.zeros((vocab_size, 300))

# 단어 집합 크기의 행과 300개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

(16, 300)

In [39]:
# 특정 단어 입력 받기

def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

# 해당 단어 임베딩 벡터 리턴, 없으면 None 리턴

In [40]:
# 단어 집합으로부터 단어를 1개씩 호출하여 word2vec_model에 해당 단어의 임베딩 벡터값이 존재하는지 확인
# 존재한다면 임베딩 행렬에 해당 단어의 인덱스 위치의 행에 임베딩 벡터의 값을 저장

for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어와 정수 인덱스를 1개씩 꺼내온다.
    temp = get_vector(word) # 단어(key) 해당되는 임베딩 벡터의 300개의 값(value)를 임시 변수에 저장
    if temp is not None: # 만약 None이 아니라면 임베딩 벡터의 값을 리턴받은 것이므로
        embedding_matrix[i] = temp # 해당 단어 위치의 행에 벡터의 값을 저장한다.

In [47]:
# 'nice'는 현재 단어 집합에서 몇 번 인덱스를 가지는지 확인
print(t.word_index)
print('단어 nice의 정수 인덱스 :', t.word_index['nice'])
print()

#  'nice'의 임베딩 벡터값 확인을 통해 맴핑 결과 확인
print(word2vec_model['nice'])

# 해당 인덱스의 임베딩 벡터 값 확인
print(embedding_matrix[1])

{'nice': 1, 'great': 2, 'best': 3, 'amazing': 4, 'stop': 5, 'lies': 6, 'pitiful': 7, 'nerd': 8, 'excellent': 9, 'work': 10, 'supreme': 11, 'quality': 12, 'bad': 13, 'highly': 14, 'respectable': 15}
단어 nice의 정수 인덱스 : 1

[ 0.15820312  0.10595703 -0.18945312  0.38671875  0.08349609 -0.26757812
  0.08349609  0.11328125 -0.10400391  0.17871094 -0.12353516 -0.22265625
 -0.01806641 -0.25390625  0.13183594  0.0859375   0.16113281  0.11083984
 -0.11083984 -0.0859375   0.0267334   0.34570312  0.15136719 -0.00415039
  0.10498047  0.04907227 -0.06982422  0.08642578  0.03198242 -0.02844238
 -0.15722656  0.11865234  0.36132812  0.00173187  0.05297852 -0.234375
  0.11767578  0.08642578 -0.01123047  0.25976562  0.28515625 -0.11669922
  0.38476562  0.07275391  0.01147461  0.03466797  0.18164062 -0.03955078
  0.04199219  0.01013184 -0.06054688  0.09765625  0.06689453  0.14648438
 -0.12011719  0.08447266 -0.06152344  0.06347656  0.3046875  -0.35546875
 -0.2890625   0.19628906 -0.33203125 -0.07128906  0.1

In [51]:
print(X_train)
print(X_train.shape)
print('시퀀스는 0을 포함한 16개')
print(y_train)
print(y_train.shape)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]
(7, 4)
시퀀스는 0을 포함한 16개
[1 0 0 1 1 0 1]
(7,)


In [48]:
# 모델 설계

model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False)
            # 16*300 // weights는 그대로 // input = 16. // False (w학습 X)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# 모델 훈련

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.6869 - acc: 0.4286
Epoch 2/100
1/1 - 0s - loss: 0.6683 - acc: 0.7143
Epoch 3/100
1/1 - 0s - loss: 0.6502 - acc: 0.7143
Epoch 4/100
1/1 - 0s - loss: 0.6327 - acc: 0.7143
Epoch 5/100
1/1 - 0s - loss: 0.6158 - acc: 0.7143
Epoch 6/100
1/1 - 0s - loss: 0.5994 - acc: 0.7143
Epoch 7/100
1/1 - 0s - loss: 0.5835 - acc: 0.8571
Epoch 8/100
1/1 - 0s - loss: 0.5682 - acc: 1.0000
Epoch 9/100
1/1 - 0s - loss: 0.5534 - acc: 1.0000
Epoch 10/100
1/1 - 0s - loss: 0.5391 - acc: 1.0000
Epoch 11/100
1/1 - 0s - loss: 0.5254 - acc: 1.0000
Epoch 12/100
1/1 - 0s - loss: 0.5121 - acc: 1.0000
Epoch 13/100
1/1 - 0s - loss: 0.4993 - acc: 1.0000
Epoch 14/100
1/1 - 0s - loss: 0.4869 - acc: 1.0000
Epoch 15/100
1/1 - 0s - loss: 0.4749 - acc: 1.0000
Epoch 16/100
1/1 - 0s - loss: 0.4634 - acc: 1.0000
Epoch 17/100
1/1 - 0s - loss: 0.4522 - acc: 1.0000
Epoch 18/100
1/1 - 0s - loss: 0.4415 - acc: 1.0000
Epoch 19/100
1/1 - 0s - loss: 0.4311 - acc: 1.0000
Epoch 20/100
1/1 - 0s - loss: 0.4210 - a